### Imports and load the data

In [1]:
from showupforhealth.utils import perform_train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 40)
import numpy as np
from imblearn.over_sampling import SMOTE
import time
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.metrics import Recall
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import seaborn as sns
from sklearn.metrics import f1_score
import keras_tuner as kt
from showupforhealth.ml_functions.predict import make_predict
from showupforhealth.utils import fit_scaler, transform_data

2023-09-17 12:12:52.299544: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load train_data and train Scaler

In [2]:
data = pd.read_csv('/Users/alessio/code/janduplessis883/data-showup/data/output-data/full_train_data.csv')
data.head(3)

,Appointment_status,Patient ID,temp,precipitation,Age,Sex,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,NDHG,SMI,IMD2023,dist_to_station,distance_to_surg,book_to_app_days,booked_by_clinician,registered_for_months,sin_week,cos_week,sin_Appointment_time,cos_Appointment_time,sin_month,cos_month,sin_day_of_week,cos_day_of_week,No_shows,Rota_ARRS,Rota_GP,Rota_HCA,Rota_Nurse,Ethnicity_Asian,Ethnicity_Black,Ethnicity_Mixed,Ethnicity_Other,Ethnicity_White
0,1.0,19580589.0,17.7,0.0,40.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22698.0,0.519419,0.816996,54.0,1.0,17.0,-0.120537,-0.992709,0.707107,-0.707107,-0.5,-0.866025,0.781831,0.623490,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,1.0,47551135.0,18.5,0.0,70.0,1.0,0.39,0.0,1.0,0.0,1.0,1.0,0.0,0.0,10169.0,0.540792,1.656309,49.0,0.0,12.0,-0.120537,-0.992709,0.500000,-0.866025,-0.5,-0.866025,0.781831,0.623490,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,1.0,19579593.0,19.4,0.0,66.0,1.0,0.03,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8243.0,0.332691,0.600495,37.0,0.0,70.0,-0.120537,-0.992709,0.258819,-0.965926,-0.5,-0.866025,0.433884,-0.900969,10.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [3]:
X = data.drop(columns=['Appointment_status'])
X.head(1)

,Patient ID,temp,precipitation,Age,Sex,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,NDHG,SMI,IMD2023,dist_to_station,distance_to_surg,book_to_app_days,booked_by_clinician,registered_for_months,sin_week,cos_week,sin_Appointment_time,cos_Appointment_time,sin_month,cos_month,sin_day_of_week,cos_day_of_week,No_shows,Rota_ARRS,Rota_GP,Rota_HCA,Rota_Nurse,Ethnicity_Asian,Ethnicity_Black,Ethnicity_Mixed,Ethnicity_Other,Ethnicity_White
0,19580589.0,17.7,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22698.0,0.519419,0.816996,54.0,1.0,17.0,-0.120537,-0.992709,0.707107,-0.707107,-0.5,-0.866025,0.781831,0.62349,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [4]:
X_cols_lst = X.columns.tolist()

In [5]:
X = X.drop(columns=['Patient ID'])
X.head(2)

,temp,precipitation,Age,Sex,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,NDHG,SMI,IMD2023,dist_to_station,distance_to_surg,book_to_app_days,booked_by_clinician,registered_for_months,sin_week,cos_week,sin_Appointment_time,cos_Appointment_time,sin_month,cos_month,sin_day_of_week,cos_day_of_week,No_shows,Rota_ARRS,Rota_GP,Rota_HCA,Rota_Nurse,Ethnicity_Asian,Ethnicity_Black,Ethnicity_Mixed,Ethnicity_Other,Ethnicity_White
0,17.7,0.0,40.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22698.0,0.519419,0.816996,54.0,1.0,17.0,-0.120537,-0.992709,0.707107,-0.707107,-0.5,-0.866025,0.781831,0.62349,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,18.5,0.0,70.0,1.0,0.39,0.0,1.0,0.0,1.0,1.0,0.0,0.0,10169.0,0.540792,1.656309,49.0,0.0,12.0,-0.120537,-0.992709,0.500000,-0.866025,-0.5,-0.866025,0.781831,0.62349,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [6]:
y = data['Appointment_status']

In [7]:
X_train, X_test, y_train, y_test = perform_train_test_split(X,y)

✅ OUTPUT: X_train, X_test, y_train, y_test
Train Set:  X_train, y_train - (476213, 36), (476213,)
 Test Set:  X_test, y_test - - (119054, 36), (119054,)


In [8]:
global_scaler = fit_scaler(X_train)

# Load ECS data and preprocess data

In [15]:
ecs_data = make_predict()

Enter Surgery Prefix: ECS

=== Preparing Appoitment Data for Prediction =================================
🌤️ Prediction: ECS - preparing appointment data for weather
👩🏻‍🦰 Appointments: 1209 🧑🏻‍🦰 Unique Patient IDs; 695
🛜 Requesting forcast from Open-Meteo Weather API 2023-08-31 - 2023-09-20
🔂 Merge weather + appointment data
✅ Successful: return df
👉 Appointment DF in: (1209, 11)
🔀 Disease Register merged with prediction data - (1209, 32)
❌ Drop NaN - Disease Register Merge = 63
⛽️ Fill NaN (0) - No Shows Merge

=== Feature Engineering =============================================================
🔂 Rename Columns
🔂 Drop deseased and deducted
🔂 Columns to Datetime
🔂 Fix Appointment Time
🔂 book_to_app_days
🔂 booked_by_clinician
🔂 Extract Rota Types
🔂 registered_for_months
🔂 Week
🔂 Month
🔂 Day of week
🔂 Convert Cyclical data
🔂 Drop Column no longer needed
🔂 Rows dropped from Rotas other than spec: 0
🔂 Rows from with Negative book_to_app_days: 0
🔂 Drop rows with Sex Unknonw & Indeterminate

In [17]:
ecs_data.head(3)

,Patient ID,temp,precipitation,Age,Sex,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,NDHG,SMI,IMD2023,dist_to_station,distance_to_surg,No_shows,book_to_app_days,booked_by_clinician,registered_for_months,sin_week,cos_week,sin_Appointment_time,cos_Appointment_time,sin_month,cos_month,sin_day_of_week,cos_day_of_week,Rota_ARRS,Rota_GP,Rota_HCA,Rota_Nurse,Ethnicity_Asian,Ethnicity_Black,Ethnicity_Mixed,Ethnicity_Other,Ethnicity_White
0,19581268,19.1,0.0,67,0,0.14,0,0,0,0,0,1,0,7626.0,0.120761,0.23344140545313152,9.0,39.0,0,19.0,-0.970942,-0.239316,0.707107,-0.707107,-1.0,-1.836970e-16,-0.781831,0.62349,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,17412911,19.8,0.0,73,0,0.28,0,1,1,1,1,0,0,6478.0,0.0722999,3.914816137524795,2.0,39.0,0,52.0,-0.970942,-0.239316,0.500000,-0.866025,-1.0,-1.836970e-16,-0.781831,0.62349,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,19579948,19.1,0.0,66,1,0.08,0,0,0,0,1,0,0,8243.0,0.376227,0.6330692363906465,0.0,39.0,0,38.0,-0.970942,-0.239316,0.707107,-0.707107,-1.0,-1.836970e-16,-0.781831,0.62349,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [18]:
ecs_data = ecs_data[X_cols_lst]
ecs_data.head(3)

,Patient ID,temp,precipitation,Age,Sex,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,NDHG,SMI,IMD2023,dist_to_station,distance_to_surg,book_to_app_days,booked_by_clinician,registered_for_months,sin_week,cos_week,sin_Appointment_time,cos_Appointment_time,sin_month,cos_month,sin_day_of_week,cos_day_of_week,No_shows,Rota_ARRS,Rota_GP,Rota_HCA,Rota_Nurse,Ethnicity_Asian,Ethnicity_Black,Ethnicity_Mixed,Ethnicity_Other,Ethnicity_White
0,19581268,19.1,0.0,67,0,0.14,0,0,0,0,0,1,0,7626.0,0.120761,0.23344140545313152,39.0,0,19.0,-0.970942,-0.239316,0.707107,-0.707107,-1.0,-1.836970e-16,-0.781831,0.62349,9.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,17412911,19.8,0.0,73,0,0.28,0,1,1,1,1,0,0,6478.0,0.0722999,3.914816137524795,39.0,0,52.0,-0.970942,-0.239316,0.500000,-0.866025,-1.0,-1.836970e-16,-0.781831,0.62349,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,19579948,19.1,0.0,66,1,0.08,0,0,0,0,1,0,0,8243.0,0.376227,0.6330692363906465,39.0,0,38.0,-0.970942,-0.239316,0.707107,-0.707107,-1.0,-1.836970e-16,-0.781831,0.62349,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
ecs_patient_id = ecs_data[['Patient ID']]
ecs_patient_id.head(3)

,Patient ID
0,19581268
1,17412911
2,19579948


In [20]:
ecs_train = ecs_data.drop(columns='Patient ID')
ecs_train.head(3)

,temp,precipitation,Age,Sex,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,NDHG,SMI,IMD2023,dist_to_station,distance_to_surg,book_to_app_days,booked_by_clinician,registered_for_months,sin_week,cos_week,sin_Appointment_time,cos_Appointment_time,sin_month,cos_month,sin_day_of_week,cos_day_of_week,No_shows,Rota_ARRS,Rota_GP,Rota_HCA,Rota_Nurse,Ethnicity_Asian,Ethnicity_Black,Ethnicity_Mixed,Ethnicity_Other,Ethnicity_White
0,19.1,0.0,67,0,0.14,0,0,0,0,0,1,0,7626.0,0.120761,0.23344140545313152,39.0,0,19.0,-0.970942,-0.239316,0.707107,-0.707107,-1.0,-1.836970e-16,-0.781831,0.62349,9.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,19.8,0.0,73,0,0.28,0,1,1,1,1,0,0,6478.0,0.0722999,3.914816137524795,39.0,0,52.0,-0.970942,-0.239316,0.500000,-0.866025,-1.0,-1.836970e-16,-0.781831,0.62349,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,19.1,0.0,66,1,0.08,0,0,0,0,1,0,0,8243.0,0.376227,0.6330692363906465,39.0,0,38.0,-0.970942,-0.239316,0.707107,-0.707107,-1.0,-1.836970e-16,-0.781831,0.62349,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [21]:
ecs_scaled = transform_data(ecs_train, global_scaler)
ecs_scaled

,temp,precipitation,Age,Sex,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,NDHG,SMI,IMD2023,dist_to_station,distance_to_surg,book_to_app_days,booked_by_clinician,registered_for_months,sin_week,cos_week,sin_Appointment_time,cos_Appointment_time,sin_month,cos_month,sin_day_of_week,cos_day_of_week,No_shows,Rota_ARRS,Rota_GP,Rota_HCA,Rota_Nurse,Ethnicity_Asian,Ethnicity_Black,Ethnicity_Mixed,Ethnicity_Other,Ethnicity_White
0,0.586124,0.0,0.632075,0.0,0.186667,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.215166,0.034173,0.007632,0.127869,0.0,0.122449,0.014529,0.380342,0.853553,0.156961,0.0,0.5,0.099031,0.801938,0.250000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.602871,0.0,0.688679,0.0,0.373333,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.177820,0.020459,0.127992,0.127869,0.0,0.346939,0.014529,0.380342,0.750000,0.071797,0.0,0.5,0.099031,0.801938,0.055556,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.586124,0.0,0.622642,1.0,0.106667,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.235239,0.106465,0.020698,0.127869,0.0,0.251701,0.014529,0.380342,0.853553,0.156961,0.0,0.5,0.099031,0.801938,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.586124,0.0,0.547170,1.0,0.040000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.025440,0.231609,0.147552,0.127869,0.0,0.299320,0.014529,0.380342,0.853553,0.156961,0.0,0.5,0.099031,0.801938,0.055556,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.602871,0.0,0.518868,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.360454,0.084748,0.159339,0.127869,0.0,0.000000,0.014529,0.380342,0.750000,0.071797,0.0,0.5,0.099031,0.801938,0.000000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141,0.662679,0.0,0.452830,1.0,0.040000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.100882,0.096696,0.101131,0.000000,0.0,0.020408,0.014529,0.380342,0.750000,0.071797,0.0,0.5,0.500000,1.000000,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1142,0.722488,0.0,0.528302,1.0,0.040000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.212792,0.073559,0.279137,0.000000,0.0,0.278912,0.014529,0.380342,0.500000,0.000000,0.0,0.5,0.500000,1.000000,0.027778,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1143,0.705742,0.0,0.358491,0.0,0.040000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.659651,0.172704,0.073803,0.000000,1.0,0.122449,0.014529,0.380342,0.017037,0.397198,0.0,0.5,0.500000,1.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1144,0.705742,0.0,0.735849,1.0,0.040000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.798887,0.144214,0.055958,0.000000,0.0,0.013605,0.014529,0.380342,0.017037,0.397198,0.0,0.5,0.500000,1.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# Train model

In [20]:
data_train = pd.read_csv('/Users/alessio/code/janduplessis883/data-showup/data/output-data/full_train_data.csv').sample(frac = 1)

In [21]:
data_DNA = data_train[data_train['Appointment_status']==0]
data_DNA.shape

(21909, 38)

In [22]:
data_NOT_DNA = data_train[data_train['Appointment_status']==1][:100000]
data_NOT_DNA.shape

(100000, 38)

In [23]:
data_balanced = pd.concat([data_DNA, data_NOT_DNA]).sample(frac = 1)
data_balanced['Appointment_status'].value_counts()

1.0    100000
0.0     21909
Name: Appointment_status, dtype: int64

In [24]:
X_t = data_balanced.drop(columns=['Appointment_status', 'Patient ID'])
y_t = data_balanced['Appointment_status']

In [25]:
print(f'The baseline accuracy if we assume that all patients will show-up is {y_t.value_counts()[1] / X_t.shape[0]}')

The baseline accuracy if we assume that all patients will show-up is 0.8202839823146774


In [26]:
X_train_t, X_test_t, y_train_t, y_test_t = perform_train_test_split(X_t, y_t)

✅ OUTPUT: X_train, X_test, y_train, y_test
Train Set:  X_train, y_train - (97527, 36), (97527,)
 Test Set:  X_test, y_test - - (24382, 36), (24382,)


In [27]:
global_scaler = fit_scaler(X_train_t)

In [28]:
X_train_scaled  = transform_data(X_train_t, global_scaler)
X_test_scaled  = transform_data(X_test_t, global_scaler)

### Split Train and Validation

In [29]:
X_train_val, X_val, y_train_val, y_val = perform_train_test_split(X_train_scaled, y_train_t)

✅ OUTPUT: X_train, X_test, y_train, y_test
Train Set:  X_train, y_train - (78021, 36), (78021,)
 Test Set:  X_test, y_test - - (19506, 36), (19506,)


# Model

In [30]:
metrics = [tf.keras.metrics.BinaryAccuracy(name='accuracy'),
          tf.keras.metrics.Recall(name='recall'),
          tf.keras.metrics.Precision(name='precision'),
          tf.keras.metrics.AUC(name='auc')]

2023-09-16 19:49:35.792270: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Model 1

In [32]:
%%time
model_one = Sequential()
es = EarlyStopping(patience=10, monitor='val_accuracy', restore_best_weights=True)

model_one.add(layers.Dense(112, activation='relu', input_dim=X_t.shape[1]))
model_one.add(layers.Dropout(0.2))
model_one.add(layers.Dense(32, activation='relu'))
model_one.add(layers.Dense(1, activation='sigmoid'))

adam = tf.keras.optimizers.Adam(learning_rate=0.001)
model_one.compile(loss='binary_crossentropy', optimizer=adam, metrics=metrics)

history_one = model_one.fit(X_train_val, y_train_val, validation_data=(X_val, y_val),
                    batch_size=32,
                    epochs=300,
                    shuffle=True,
                    verbose=0,
                    callbacks=[es])

results = model_one.evaluate(X_test_scaled, y_test_t)
print(f'Model {model_one.metrics_names[0]} is {results[0]}')
print(f'Model {model_one.metrics_names[1]} is {results[1]}')
print(f'Model {model_one.metrics_names[2]} is {results[2]}')
print(f'Model {model_one.metrics_names[3]} is {results[3]}')
print(f'Model {model_one.metrics_names[4]} is {results[4]}')

762/762 [==============================] - 1s 1ms/step - loss: 0.2763 - accuracy: 0.8654 - recall: 0.9387 - precision: 0.9011 - auc: 0.9112
Model loss is 0.27626729011535645
Model accuracy is 0.8653514981269836
Model recall is 0.9386671781539917
Model precision is 0.9010862112045288
Model auc is 0.91123366355896
CPU times: user 1min 27s, sys: 15.2 s, total: 1min 42s
Wall time: 1min 2s


# Predict with saved model

### Load saved model

In [124]:
from tensorflow.keras.models import load_model
model_two = load_model('model_two.h5')

In [125]:
model_two_pred = model_two.predict(ecs_scaled)
model_two_pred = (model_two_pred > 0.5).astype(np.float32)
model_two_pred = pd.DataFrame(model_two_pred).rename(columns={0:'PREDICTION'})
model_two_pred

36/36 [==============================] - 0s 1ms/step


,PREDICTION
0,0.0
1,0.0
2,1.0
3,0.0
4,1.0
...,...
1141,1.0
1142,1.0
1143,1.0
1144,1.0


In [141]:
pred_df = model_two_pred.join(ecs_patient_id)
pred_df['PREDICTION'].value_counts()

1.0    898
0.0    248
Name: PREDICTION, dtype: int64

### Load actual ECS data

In [134]:
ecs_with_app = pd.read_csv('/Users/alessio/code/janduplessis883/data-showup/data/predict/Original with Appointment Status/ECS_Predict.csv')

In [140]:
ecs_with_app = ecs_with_app[['Appointment status', 'Patient ID']]
ecs_with_app['Appointment status'].value_counts()

Finished          1162
Did Not Attend      38
Arrived              5
In Progress          4
Name: Appointment status, dtype: int64

In [138]:
ecs_final = pred_df.merge(ecs_with_app,on='Patient ID', how='left')
ecs_final

,PREDICTION,Patient ID,Appointment status
0,0.0,19581268,Did Not Attend
1,0.0,19581268,Finished
2,0.0,17412911,Did Not Attend
3,0.0,17412911,Finished
4,0.0,17412911,Finished
...,...,...,...
2915,1.0,17407944,Finished
2916,1.0,39650996,Finished
2917,1.0,19579038,Finished
2918,1.0,19579038,Finished


In [117]:
df1 = pd.DataFrame(data={'col_1': [1, 2], 'col_2': [3, 4]})
df1

,col_1,col_2
0,1,3
1,2,4


In [118]:
df2 = pd.DataFrame(data={'col_1': [1, 2], 'col_3': [7, 8]})
df2

,col_1,col_3
0,1,7
1,2,8


In [120]:
df3 = df1.merge(df2, how='left', on='col_1')
df3

,col_1,col_2,col_3
0,1,3,7
1,2,4,8
